## 수요예측모델 Class, Def로 구현하기

In [0]:
import pandas as pd
import numpy as np
import dateutil
from dateutil.parser import parse
import datetime
from datetime import timedelta

class Customer:
    def __init__(self, data, name):
        self.name = name
        self.receipt = data[data['주문자ID']==self.name].sort_values(by=['주문일시'])
        
    # 총 몇 번 주문했는지
    def order_count(self):
        return self.receipt.shape[0]

    # 마지막 주문은 언제인지
    def last_order(self):
        df_cst = self.receipt[self.receipt['주문자ID']==self.name]
        last_order = df_cst['주문일시'].iloc[-1]
        print('{}년 {}월 {}일'.format(last_order.year, last_order.month, last_order.day))
        
    # 특정 기간동안 샀던 품목들과 평균구매주기를 반환
    def purchase_product(self, yyyymmdd = pd.Timestamp.now(), month = 3):
        if type(month) != int:
            raise ValueError("month must be a int")
        self.month = month
        self.start_point = yyyymmdd
        if type(yyyymmdd) is not pd.Timestamp:
            self.start_point = datetime.datetime(year=int(yyyymmdd[:4]), month=int(yyyymmdd[4:6]), day=int(yyyymmdd[6:]))
        elif type(yyyymmdd) is pd.Timestamp:
            self.start_point = yyyymmdd
            
        back_point = self.start_point-timedelta(days=self.month*30)
        print('시작일시 :', back_point)
        print('종료일시 :', self.start_point)
        receipt_limited = self.receipt[self.receipt['주문일시']>back_point]
        receipt_limited = receipt_limited[receipt_limited['주문일시']<self.start_point]
        self.receipt_limited = receipt_limited

        print('기간 내 결제금액 :', self.receipt_limited.drop_duplicates(['주문일시'])['총 결제금액'].sum())
        product_list = [product for product in list(receipt_limited['세부분류사이즈'].unique()) if type(product) is str]  
        product_list = list(set(product_list))
        
        list_product_average_order_period = []
        list_product_total_using_period = []
        for product in product_list:
            list_collect_product_order_period = []
            df_specific_product = receipt_limited[receipt_limited['세부분류사이즈']==product].drop_duplicates(['주문일시']).sort_values(by='주문일시') # 해당 품목만 있는 데이터프레임 만들어주기
            
            # 먼저 상품구매경험이 2회 이상인 품목들의 구매주기 모아주기
            if (df_specific_product.shape[0] > 1):
                for i in range(df_specific_product.shape[0]-1):
                    product_order_period = df_specific_product['주문일시'].iloc[i+1] - df_specific_product['주문일시'].iloc[i]
                    if product_order_period.days == 0:    # 하루 내 여러 차례 주문한 것들은 한 번 주문한 것으로 간주
                        list_collect_product_order_period.append([product, 0]) # 구매주기를 품목과 같이 append
                    else:
                        list_collect_product_order_period.append([product, product_order_period]) # 구매주기를 품목과 같이 append
                        
            # 상품구매경험이 1회밖에 없는 사람들의 구매주기는 0으로 계산
            else:  
                for i in range(df_specific_product.shape[0]):
                    list_collect_product_order_period.append([product, 0])

            # 구매주기의 평균 구하기
            total_timedelta = datetime.timedelta(days=0, seconds=0)
            for i in list_collect_product_order_period:
                if type(i[1]) is timedelta:
                    total_timedelta = total_timedelta + i[1]
            average_order_period = round(total_timedelta.days/len(list_collect_product_order_period),2)
            list_product_average_order_period.append([product, average_order_period])    # 품목별 평균구매주기 구했음

            # 상품 누적사용일 구하기
            if (df_specific_product.shape[0] > 1):
                product_1st_end_period = (df_specific_product['주문일시'].iloc[-1] - df_specific_product['주문일시'].iloc[0]).days
                list_product_total_using_period.append([product, product_1st_end_period + average_order_period])
            else:
                list_product_total_using_period.append([product, 0])
        
        # 전체 사용주기에서 제품이랑 누적사용일 zip해줘서 dictionary로 만들고
        product = []
        days = []
        for i in list_product_total_using_period:
            product.append(i[0])
            days.append(i[1])
        day_dict = dict(zip(product, days))

        df_pivot = self.receipt_limited.pivot_table(index='주문연월', columns='세부분류사이즈',
                               values='총주문량', aggfunc=np.sum)
        df_pivot = df_pivot.T.fillna(0).reset_index()
        df_pivot['총합'] = df_pivot.sum(axis=1)
        df_pivot['전체사용일'] = df_pivot['세부분류사이즈'].map(day_dict)
        df_pivot['하루당 사용량'] = round((df_pivot['총합'] / df_pivot['전체사용일']),2)
        final_pivot = df_pivot[['세부분류사이즈','총합','전체사용일','하루당 사용량']]
        final_pivot = final_pivot.dropna(axis=0)
        final_pivot['하루당 사용량'] = final_pivot['하루당 사용량'].replace(np.inf, 0)
        final_pivot = final_pivot.sort_values(by=['총합'], ascending=False)
        self.final_pivot = final_pivot
        return final_pivot

    # 하루 수요량 계산
    def daily_demand(self, yyyymmdd = pd.Timestamp.now(), month = 3):
        self.yyyymmdd = yyyymmdd
        self.month = month
        final_pivot = self.purchase_product(self.yyyymmdd, self.month)
        baby_data = dict(zip(final_pivot['세부분류사이즈'].values, final_pivot['하루당 사용량'].values))
        baby_data['주문자ID'] = self.name
        baby_df = pd.DataFrame(baby_data, columns=list(baby_data.keys()), index=[0])
        baby_df = baby_df.dropna(axis=0)
        cols = list(baby_df.columns)
        cols = [cols[-1]]+cols[:-1]
        return baby_df[cols]
    
    # 회원등급 파악하기
    def membership(self):
        if self.receipt['군집_5'].iloc[0] == 2:
            return 'Level.5 대상인'
        if self.receipt['군집_5'].iloc[0] == 4:
            return 'Level.4 장사 잘 되는 중소상인'
        if self.receipt['군집_5'].iloc[0] == 1:
            return 'Level.3 중소상인'
        if self.receipt['군집_5'].iloc[0] == 3:
            return 'Level.2 영세상인'
        if self.receipt['군집_5'].iloc[0] == 0:
            return 'Level.1 병아리'
        if self.receipt['군집_5'].iloc[0] == None:
            return 'Not Royal'
        
    # 구매주기 파악하기
    def period_cycle(self):
        period_list = []
        df = self.receipt.drop_duplicates(['주문일시'])
        for i in range(df.shape[0]-1):
            period = df['주문일시'].iloc[i+1] - df['주문일시'].iloc[i]
            if period > datetime.timedelta(days=1):
                period_list.append(period)
        total = datetime.timedelta(days=0)
        for i in period_list:
            total = total + i
        period_cycle = total/len(period_list)
        return period_cycle

### 수요예측하기

In [0]:
## 적용해보기
mother_columns = list(pck_1462['세부분류사이즈'].unique())
mother_columns.insert(0, '주문자ID')
mother_df = pd.DataFrame(columns=mother_columns)

for customer_id in tqdm(list(pck_1462['주문자ID'].unique())):
    ex = Customer(pck, customer_id)
    baby_df = ex.daily_demand(yyyymmdd = '20190501',month = 5)
    mother_df = pd.concat([mother_df, baby_df])

# column 순서 이쁘게 정리하기
mother_df = mother_df[[ '주문자ID','감자탕용기_2200ml', '감자탕용기_2700ml', '감자탕용기_3200ml', '고강도미니탕용기_미니', '고무장갑', '군만두용기_1칸', '냄비_2750ml', '냄비뚜껑_大', '냉면용기_中', '냉면용기_大', '다용도컵_150ml', '다용도컵_165ml', '다용도컵_200ml', '다용도컵_250ml', '다용도컵_270ml', '다용도컵_300ml', '다용도컵_370ml', '다용도컵_400ml', '다용도컵_50ml', '다용도컵_600ml', '다용도컵_70ml', '다용도컵_80ml', '단무지', '도시락용기(돈까스)_5칸', '도시락용기(돈까스)_中', '도시락용기_4칸 A', '도시락용기_4칸 B', '도시락용기_5칸', '도시락용기_6칸 02', '도시락용기_6칸 L1', '도시락용기_7칸', '도시락용기_8칸', '랩칼', '미니탕용기_미니', '반찬용기_104_1칸', '반찬용기_105_1칸', '반찬용기_201_1칸', '반찬용기_202-2_2칸', '반찬용기_202_1칸', '반찬용기_203_1칸', '반찬용기_204_1칸', '반찬용기_211_1칸', '반찬용기_D-363', '반찬용기_D-364_4칸', '반찬용기_D-365_5칸', '비닐봉투(맛난거)_中', '비닐봉투(맛난거)_大', '비닐봉투(맛난거)_小', '비닐봉투(맛난거)_특大', '비닐봉투(무지)_中', '비닐봉투(무지)_大', '비닐봉투(무지)_小', '비닐봉투(무지)_특大', '비닐봉투(배달중)_中',
 '비닐봉투(배달중)_大', '비닐봉투(배달중)_小', '비닐봉투(배달중)_특大', '사각찬용기_4칸', '샐러드_1칸(550ml)', '샐러드_3칸(150ml)', '샐러드_4칸(100ml)', '수세미', '실링기계', '실링용기_1000ml', '실링용기_1150ml', '실링용기_1350ml', '실링용기_1500ml', '실링용기_2000ml', '실링용기_200ml', '실링용기_2200ml', '실링용기_230ml', '실링용기_3-5A', '실링용기_400ml', '실링용기_500ml', '실링용기_5칸', '실링용기_600ml', '실링용기_650ml', '실링용기_700ml', '실링용기_84-2A', '실링용기_900ml', '실링필름_150mm x 200 m', '실링필름_160mm x 200m', '실링필름_190mm x 200m', '실링필름_200mm x 200m', '실링필름_245mm  x 200m', '아이스컵 뚜껑_돔형_구멍', '아이스컵 뚜껑_돔형_노구멍', '아이스컵 뚜껑_평형_민자형_십자구멍', '아이스컵_12ounce', '아이스컵_14ounce', '아이스컵_16ounce', '아이스컵_24ounce', '아이스컵_돔형_구멍', '우동용기_中', '우동용기_大', '우동용기_특大', '원형용기_150ml', '원형용기_350ml', '원형용기_450ml', '원형용기_470ml', '원형용기_大', '원형접시_大', '원형접시_小', '원형접시_미니', '위생랩_2호', '위생롤백_2호', '위생롤백_5호', '위생장갑', '일회용 숟가락', '일회용 젓가락_198mm', '일회용 포크_160mm', '종이호일_1호', '종이호일_2호',
 '죽용기_1050ml', '죽용기_1150ml', '죽용기_400ml', '죽용기_550ml', '죽용기_750ml', '죽용기_920ml', '중식소스용기', '중화면용기 뚜껑', '중화면용기_中', '중화면용기_大', '중화면용기_小', '지퍼백_M', '짬짜면_2칸', '찜용기(사각)_2000ml', '찜용기(사각)_2500ml', '찜용기(사각)_3200ml', '찜용기(원)_2500ml', '찜용기(원)_3500ml', '찜용기(원)_4000ml', '찜용기(원)_4500ml', '찜용기(타원)_1800ml', '찜용기(타원)_2500ml', '찜용기(타원)_3000ml', '찬용기_250ml', '찬용기_350ml', '찬용기_450ml', '타원접시', '탕용기_195_大', '탕용기_195_小', '탕용기_210_大', '탕용기_210_小', '탕용기_225_1800ml', '탕용기_225_2400ml', '탕용기_225_3000ml', '탕용기_240_5000ml', '탕용기_270_6500ml', '행주']]

mother_df = mother_df.fillna(0) 

mother_df

## 지도시각화

In [0]:
korea = [36.39, 127.28]
map_korea = folium.Map(location=korea, zoom_start=7)

# 군집 0
cluster_0 = data[data['군집_5']==0]
# 군집 1
cluster_1 = data[data['군집_5']==1]
# 군집 2
cluster_2 = data[data['군집_5']==2]
# 군집 3
cluster_3 = data[data['군집_5']==3]
# 군집 4
cluster_4 = data[data['군집_5']==4]

# 군집 0 찍기
for customer in tqdm(list(cluster_0['주문자ID'].unique())):
    temp = cluster_0[cluster_0['주문자ID']==customer]
    folium.Marker([temp['위도'].values[0], temp['경도'].values[0]], popup=customer, icon=folium.Icon(icon='cloud', color = "black")).add_to(map_korea)

# 군집 1 찍기
for customer in tqdm(list(cluster_1['주문자ID'].unique())):
    temp = cluster_1[cluster_1['주문자ID']==customer]
    folium.Marker([temp['위도'].values[0], temp['경도'].values[0]], popup=customer, icon=folium.Icon(icon='cloud', color = "green")).add_to(map_korea)
    
# 군집 2 찍기
for customer in tqdm(list(cluster_2['주문자ID'].unique())):
    temp = cluster_2[cluster_2['주문자ID']==customer]
    folium.Marker([temp['위도'].values[0], temp['경도'].values[0]], popup=customer, icon=folium.Icon(icon='cloud', color = "purple")).add_to(map_korea)
    
# 군집 3 찍기
for customer in tqdm(list(cluster_3['주문자ID'].unique())):
    temp = cluster_3[cluster_3['주문자ID']==customer]
    folium.Marker([temp['위도'].values[0], temp['경도'].values[0]], popup=customer, icon=folium.Icon(icon='cloud', color = 'blue')).add_to(map_korea)
    
# 군집 4 찍기
for customer in tqdm(list(cluster_4['주문자ID'].unique())):
    temp = cluster_4[cluster_4['주문자ID']==customer]
    folium.Marker([temp['위도'].values[0], temp['경도'].values[0]], popup=customer, icon=folium.Icon(icon='cloud', color = "red")).add_to(map_korea)    

map_korea.save('지도시각화_프렌즈 등급 이상.html')

## 1.5km 내 고객 수 구하기

In [0]:
## 거리 구하는 함수
import numbers
import math

class GeoUtil:
    """
    Geographical Utils
    """
    @staticmethod
    def degree2radius(degree):
        return degree * (math.pi/180)
    
    @staticmethod
    def get_harversion_distance(x1, y1, x2, y2, round_decimal_digits=5):
        """
        경위도 (x1,y1)과 (x2,y2) 점의 거리를 반환
        Harversion Formula 이용하여 2개의 경위도간 거래를 구함(단위:Km)
        """
        if x1 is None or y1 is None or x2 is None or y2 is None:
            return None
        assert isinstance(x1, numbers.Number) and -180 <= x1 and x1 <= 180
        assert isinstance(y1, numbers.Number) and  -90 <= y1 and y1 <=  90
        assert isinstance(x2, numbers.Number) and -180 <= x2 and x2 <= 180
        assert isinstance(y2, numbers.Number) and  -90 <= y2 and y2 <=  90

        R = 6371 # 지구의 반경(단위: km)
        dLon = GeoUtil.degree2radius(x2-x1)    
        dLat = GeoUtil.degree2radius(y2-y1)

        a = math.sin(dLat/2) * math.sin(dLat/2) \
            + (math.cos(GeoUtil.degree2radius(y1)) \
              *math.cos(GeoUtil.degree2radius(y2)) \
              *math.sin(dLon/2) * math.sin(dLon/2))
        b = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
        return round(R * b, round_decimal_digits)

    @staticmethod
    def get_euclidean_distance(x1, y1, x2, y2, round_decimal_digits=5):        
        """
        유클리안 Formula 이용하여 (x1,y1)과 (x2,y2) 점의 거리를 반환
        """
        if x1 is None or y1 is None or x2 is None or y2 is None:
            return None
        assert isinstance(x1, numbers.Number) and -180 <= x1 and x1 <= 180
        assert isinstance(y1, numbers.Number) and  -90 <= y1 and y1 <=  90
        assert isinstance(x2, numbers.Number) and -180 <= x2 and x2 <= 180
        assert isinstance(y2, numbers.Number) and  -90 <= y2 and y2 <=  90

        dLon = abs(x2-x1) # 경도 차이
        if dLon >= 180:   # 반대편으로 갈 수 있는 경우
            dLon -= 360   # 반대편 각을 구한다
        dLat = y2-y1      # 위도 차이
        return round(math.sqrt(pow(dLon,2)+pow(dLat,2)),round_decimal_digits)

In [0]:
## 고객간의 거리 구해주기
cst_total_list = list(df_cst['주문자ID'].unique())
list_id = []
list_num_of_cst = []
for cst in tqdm(cst_total_list):
    cri_cst_lon = df_cst[df_cst['주문자ID']==cst]['경도'].values[0]
    cri_cst_lat = df_cst[df_cst['주문자ID']==cst]['위도'].values[0]
    temp = cst_total_list.copy()
    temp.remove(cst)
    list_distance_collection = []
    list_id.append(cst)

    # 거리 구하기
    for i in temp:
        i_lon = df_cst[df_cst['주문자ID']==i]['경도'].values[0]
        i_lat = df_cst[df_cst['주문자ID']==i]['위도'].values[0]
        list_distance_collection.append(GeoUtil.get_harversion_distance(cri_cst_lon, cri_cst_lat, i_lon, i_lat))
    num_of_cst = len([dist for dist in list_distance_collection if dist<1.5])
    list_num_of_cst.append(num_of_cst)